# 소프트웨어 재판매 이익예측 form Tayko by KH

## 라이브러리 임포트

In [27]:
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt

import dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

%matplotlib inline


## 데이타프레임 생성 및 변수 정리

In [28]:
soft_df = dmba.load_data('tayko.csv')
soft_df = soft_df.iloc[0:2000]

In [29]:
print(soft_df.columns.values[20])

Web order


In [30]:
# 컬럼 이름 바꾸기
soft_df.columns.values[20] = 'Web_order'
soft_df.columns.values[21] = 'Gender_male'


In [31]:
print(soft_df)

      sequence_number  US  source_a  source_c  source_b  source_d  source_e  \
0                   1   1         0         0         1         0         0   
1                   2   1         0         0         0         0         1   
2                   3   1         0         0         0         0         0   
3                   4   1         0         1         0         0         0   
4                   5   1         0         1         0         0         0   
...               ...  ..       ...       ...       ...       ...       ...   
1995             1996   1         0         0         0         0         0   
1996             1997   1         0         0         0         0         0   
1997             1998   1         0         0         0         0         0   
1998             1999   1         0         0         0         0         0   
1999             2000   1         0         0         0         0         0   

      source_m  source_o  source_h  ...  source_x  

In [32]:
# 컬럼 삭제 
soft_df.drop(['source_a','source_c','source_b','source_d','source_e'], axis=1, inplace=True)

In [33]:
# 컬럼 삭제 
soft_df.drop(['source_m','source_o','source_h','source_r','source_s'], axis=1, inplace=True)

In [34]:
# 컬럼 삭제 
soft_df.drop(['source_t','source_u','source_p','source_x','source_w'], axis=1, inplace=True)

In [35]:
# 컬럼 삭제 
soft_df.drop(['1st_update_days_ago','Purchase'], axis=1, inplace=True)

In [36]:
print(soft_df)

      sequence_number  US  Freq  last_update_days_ago  Web_order  Gender_male  \
0                   1   1     2                  3662          1            0   
1                   2   1     0                  2900          1            1   
2                   3   1     2                  3883          0            0   
3                   4   1     1                   829          0            1   
4                   5   1     1                   869          0            0   
...               ...  ..   ...                   ...        ...          ...   
1995             1996   1     1                  1701          1            0   
1996             1997   1     1                  2633          1            1   
1997             1998   1     0                  3394          0            0   
1998             1999   1     1                   253          0            1   
1999             2000   1     1                  1261          0            0   

      Address_is_res  Spend

In [37]:
#null data 확인
soft_df.isnull()

,sequence_number,US,Freq,last_update_days_ago,Web_order,Gender_male,Address_is_res,Spending
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
1995,False,False,False,False,False,False,False,False
1996,False,False,False,False,False,False,False,False
1997,False,False,False,False,False,False,False,False
1998,False,False,False,False,False,False,False,False


In [38]:
#null data 확인
soft_df.isnull().sum()

sequence_number         0
US                      0
Freq                    0
last_update_days_ago    0
Web_order               0
Gender_male             0
Address_is_res          0
Spending                0
dtype: int64

In [39]:
# 만약 null 값이 있을 경우 삭제방법
# soft_df = soft_df.dropna()   null값이 있는 모든 행 삭제
# 또는 soft_df = df.dropna(axis=1)   null값이 있는 열(coulumn) 모두 제거


In [40]:
print(soft_df.describe())

       sequence_number           US         Freq  last_update_days_ago  \
count      2000.000000  2000.000000  2000.000000           2000.000000   
mean       1000.500000     0.824500     1.417000           2155.101000   
std         577.494589     0.380489     1.405738           1141.302846   
min           1.000000     0.000000     0.000000              1.000000   
25%         500.750000     1.000000     1.000000           1133.000000   
50%        1000.500000     1.000000     1.000000           2280.000000   
75%        1500.250000     1.000000     2.000000           3139.250000   
max        2000.000000     1.000000    15.000000           4188.000000   

         Web_order  Gender_male  Address_is_res    Spending  
count  2000.000000  2000.000000     2000.000000  2000.00000  
mean      0.426000     0.524500        0.221000   102.62500  
std       0.494617     0.499524        0.415024   186.78261  
min       0.000000     0.000000        0.000000     0.00000  
25%       0.000000     

### 범주별 소비금액의 평균과 표준편차

In [41]:
gor_df = soft_df.groupby('US').agg({'Spending':['mean','std']})
print(gor_df)

      Spending            
          mean         std
US                        
0   101.216524  174.844401
1   102.924803  189.275664


In [42]:
print(soft_df['Freq'])

0       2
1       0
2       2
3       1
4       1
       ..
1995    1
1996    1
1997    0
1998    1
1999    1
Name: Freq, Length: 2000, dtype: int64


In [43]:
dum = pd.get_dummies(soft_df['Freq'])
print(dum)

      0   1   2   3   4   5   6   7   8   9   10  11  12  13  15
0      0   0   1   0   0   0   0   0   0   0   0   0   0   0   0
1      1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2      0   0   1   0   0   0   0   0   0   0   0   0   0   0   0
3      0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
4      0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
1995   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
1996   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
1997   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
1998   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
1999   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0

[2000 rows x 15 columns]


In [45]:
gor_df = soft_df.groupby(['Freq'])['Spending'].agg(['mean','std'])
print(gor_df)

             mean         std
Freq                         
0        0.000000    0.000000
1       66.322476  104.424412
2      123.479714  151.509696
3      234.993243  226.259754
4      306.061224  165.153642
5      459.862069  270.661719
6      556.750000  344.195009
7      642.125000  520.126478
8      933.500000  324.925120
9      870.500000  433.828307
10    1199.000000   21.213203
11    1334.000000  151.320851
12    1320.500000  177.483802
13    1443.000000         NaN
15    1133.000000         NaN


In [46]:
gor_df = soft_df.groupby(['last_update_days_ago'])['Spending'].agg(['mean','std'])
print(gor_df)

                            mean         std
last_update_days_ago                        
1                     109.000000         NaN
7                     129.000000         NaN
9                     196.000000         NaN
14                    303.000000         NaN
15                     71.000000  100.409163
...                          ...         ...
4065                   64.750000   58.987993
4096                   75.666667   93.681731
4127                   17.500000   24.748737
4157                   75.666667   95.516840
4188                   88.000000   76.374079

[940 rows x 2 columns]


In [47]:
gor_df = soft_df.groupby(['Web_order'])['Spending'].agg(['mean','std'])
print(gor_df)

                 mean         std
Web_order                        
0           82.902439  173.417088
1          129.199531  200.463840


In [48]:
gor_df = soft_df.groupby(['Gender_male'])['Spending'].agg(['mean','std'])
print(gor_df)

                   mean        std
Gender_male                       
0            107.339642  190.83233
1             98.350810  183.02006


In [49]:
gor_df = soft_df.groupby(['Address_is_res'])['Spending'].agg(['mean','std'])
print(gor_df)

                      mean         std
Address_is_res                        
0               105.306162  199.521159
1                93.174208  132.204281


### 연속형 변수들에 대한 산점도(2개)작성하여 소비금액과의 관계 탐색

  
  
  ## 모델 생성
predictiors : 독립변수로 사용할 변수 선택  
outcome : 종속변수로 사용할 변수 선택

In [50]:
predictors = ['US', 'Freq', 'last_update_days_ago', 'Web_order', 'Gender_male', 'Address_is_res']   #독립변수로 사용될 Coloum
outcome = 'Spending'    # 종속변수

# partition data 
X = pd.get_dummies(soft_df[predictors], drop_first=True)   #범주형 변수를 더미변수(이진수로 표현되는 변수, 0과 1로 나타냄)로 변환
y = soft_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1) # 학습데이터와 검증데이터로 분할

soft_lm = LinearRegression()    #선형 회귀모델 클래스를 사용하여 soft_lm 객체를 생성
soft_lm.fit(train_X, train_y)   #soft_lm 객체를 학습, 학습후 soft_lm을을 사용하여 새로운 입력 데이터에 대한 예측 수행

# print coefficients
print('intercept ', soft_lm.intercept_)    # 학습된 절편출력
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': soft_lm.coef_}))  #pandas dataframe 형식으로 독립변수에 대한 coefficient 나타냄 
 
# print performance measures
regressionSummary(train_y, soft_lm.predict(train_X)) #  모델의 성능평가 및 요약, Train y값과 예측된 y값을 받아 예측 성능에 대한 통계치와 정보를 출력

intercept  10.17629741458967
              Predictor  coefficient
0                    US    -4.620293
1                  Freq    91.274450
2  last_update_days_ago    -0.010374
3             Web_order    18.628731
4           Gender_male    -9.111366
5        Address_is_res   -75.815354

Regression statistics

               Mean Error (ME) : 0.0000
Root Mean Squared Error (RMSE) : 125.9999
     Mean Absolute Error (MAE) : 79.4772


RMSE : 실제 값과 예측 값의 차이의 제곱값의 평균의 제곱근
MAE : 예측값과 실제 값의 차이를 평균한 값

In [51]:
pred_y = soft_lm.predict(train_X)   # soft_lm(학습된 선형 회귀모델)을 사용하여 입력데이타(train_X)에 대한 예측 값을 계산하여 pred_y에 저장

print('adjusted r2 : ', adjusted_r2_score(train_y, pred_y, soft_lm))   : 결과 R2 값이 높을 수록 모델의 설명력이 높단고 판단할 수 있다.
print('AIC : ', AIC_score(train_y, pred_y, soft_lm))
print('BIC : ', BIC_score(train_y, pred_y, soft_lm))

SyntaxError: illegal target for annotation (397636381.py, line 3)

In [ ]:
# Use predict() to make predictions on a new set
soft_lm_pred = soft_lm.predict(valid_X)

result = pd.DataFrame({'Predicted': soft_lm_pred, 'Actual': valid_y,
                       'Residual': valid_y - soft_lm_pred})
print(result.head(20))

# Compute common accuracy measures
regressionSummary(valid_y, soft_lm_pred)


In [ ]:
# MPE, MAPE 는 왜 출력되지 않지?

In [ ]:
soft_lm_pred = soft_lm.predict(valid_X)
all_residuals = valid_y - soft_lm_pred

# Determine the percentage of datapoints with a residual in [-1406, 1406] = approx. 75\%   ??? data에 맞게 수정해야 함
print(len(all_residuals[(all_residuals > -1406) & (all_residuals < 1406)]) / len(all_residuals))

ax = pd.DataFrame({'Residuals': all_residuals}).hist(bins=25)

plt.tight_layout()
plt.show()

In [ ]:
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # we negate as score is optimized to be as low as possible
    return -adjusted_r2_score(train_y, pred_y, model)

allVariables = train_X.columns
results = exhaustive_search(allVariables, train_model, score_model)

data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)
pd.set_option('display.width', 100)
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))
pd.reset_option('display.width')

In [ ]:
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = backward_elimination(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

In [ ]:
regressionSummary(valid_y, best_model.predict(valid_X[best_variables]))

In [ ]:
# The initial model is the constant model - this requires special handling
# in train_model and score_model
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

In [ ]:
best_model, best_variables = stepwise_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

In [ ]:
lasso = Lasso(normalize=True, alpha=1)
lasso.fit(train_X, train_y)
regressionSummary(valid_y, lasso.predict(valid_X))

lasso_cv = LassoCV(normalize=True, cv=5)
lasso_cv.fit(train_X, train_y)
regressionSummary(valid_y, lasso_cv.predict(valid_X))
print('Lasso-CV chosen regularization: ', lasso_cv.alpha_)
print(lasso_cv.coef_)

ridge = Ridge(normalize=True, alpha=1)
ridge.fit(train_X, train_y)
regressionSummary(valid_y, ridge.predict(valid_X))

bayesianRidge = BayesianRidge(normalize=True)
bayesianRidge.fit(train_X, train_y)
regressionSummary(valid_y, bayesianRidge.predict(valid_X))
print('Bayesian ridge chosen regularization: ', bayesianRidge.lambda_ / bayesianRidge.alpha_)

In [ ]:
linearRegression = LinearRegression(normalize=True).fit(train_X, train_y)
regressionSummary(valid_y, linearRegression.predict(valid_X))

In [ ]:
pd.DataFrame({'features': train_X.columns, 'linear regression': linearRegression.coef_, 
              'lassoCV': lasso_cv.coef_, 'bayesianRidge': bayesianRidge.coef_})

In [ ]:
# run a linear regression of Price on the remaining 11 predictors in the training set
train_df = train_X.join(train_y)

predictors = train_X.columns
formula = 'Spending ~ ' + ' + '.join(predictors)

soft_lm = sm.ols(formula=formula, data=train_df).fit()
print(soft_lm.summary())

## 결과값이 0인 데이터 삭제하여 재분석
### 본 데이타는 미국 거주 및 